# 第8回 その2: k-means
5教科テストデータに対して k-means を行い自動クラス分類を行ってみましょう。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/08"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/08/
# 現在のフォルダの中身を表示
%ls

`gokyouka.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込み
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`gokyouka.csv` を読み込みます。  
このデータは第2回のレポート課題で使用した，5教科のテストの点数を示したデータ(report02_input.csv)です。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('gokyouka.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

## ステップ2: 主成分分析によるデータの2次元への圧縮  
今回はk-meansの途中経過をプロットしながら動作確認をします。  
5教科データのままではプロットできませんので，主成分分析を用いて5教科データを2次元に圧縮しておきます。  
主成分分析については第5回で解説していますので参照してください。  

**補足**  
今回は途中結果を可視化するため主成分分析を用いて2次元に圧縮してから k-means を使っていますが，  
本来はk-meansに主成分分析は必須ではありません。  
5次元データのままk-meansを実施して問題はありません。  
ただし，元データの次元数が多すぎる場合は，計算時間がかかるので，主成分分析を使って次元削減を行ってからk-meansを実施するケースは多々あります（この場合は2次元以上の次元数に圧縮しても構いません）。

In [ ]:
# データの標準化
X_norm = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 分散共分散行列の計算
cov = np.cov(X_norm.T, bias=True)
# 固有値分解
eig_val, eig_vec =np.linalg.eig(cov)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec_pca = eig_vec[:,order]

# 第二主成分までの固有ベクトルを用いて次元圧縮
X_pca = np.dot(X_norm, eig_vec_pca[:,:2])

# 二次元データのプロット
plt.figure(figsize=(10,6))
plt.scatter(X_pca[:,0], X_pca[:,1], color='k')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.show()

主成分分析によって2次元圧縮したデータが `X_pca` に格納されました。  
以降は `X_pca` に対して k-means を行います。  

## ステップ3: セントロイド初期値の設定  
ここからが k-means の処理になります。  
ここでは分割するクラスの数 $K=4$ とします。  

まずは各クラスの中心データ（セントロイド）の初期値をランダムに決めます。 
ここでは全データからランダムに $K=4$ 個取り出し，取り出した4個のデータの値をセントロイドの初期値とします。  

ランダムにデータを取り出すために，ここではデータをランダムに並び替えた後，先頭から$K=4$個取り出すという方法を取っています。

In [ ]:
#クラス数
K = 4

# 乱数シードの設定(ここではシードを5としていますが，何でも良いです)
np.random.seed(5)

# 0～99 をランダムに並び替える
print('shuffled indexes')
shuffle_index = np.random.permutation(np.arange(num_samples))
print(shuffle_index)

# 並び替えた数の先頭 K個を取り出して、セントロイドの初期値とする。
centroid_index = shuffle_index[:K]
print('initial centroid index')
print('  ' + str(centroid_index))
initial_centroid = X_pca[centroid_index,:]

# 各セントロイドのデータ（5教科点数）を表示
print('Centroids')
print(initial_centroid)

上記の処理では，まず0～99の整数をランダムに並び替えています。  
その後先頭の $K=4$ 個を取り出しています。  
取り出した結果は [66, 32, 46, 28] となっています。  
(同じ乱数シードを使っていれば，皆さんの実行結果も同じ値になっているはずですが，もし別の値になっている場合は教えてください。)  

よって，66番目，32番目，46番目，28番目のデータを取ってきて，その値をセントロイドの初期値とします。  
ただし主成分分析を行った後のデータ `X_pca` に対して処理を送るので，セントロイドの次元数は5ではなく2である点に注意してください。

各データ `X_pca` とセントロイド初期値 `initial_centroid` をプロットしてみましょう。  
（66，32，46，28番がセントロイドになっていることを示すため，プロット結果にサンプル番号を表示しています。）

In [ ]:
# 色のセット
colors = ['r', 'b', 'g', 'c', 'm', 'y']

plt.figure(figsize=(10,6))
# データ全体のプロット
plt.scatter(X_pca[:,0], X_pca[:,1], color='grey', label='data')
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  plt.scatter(initial_centroid[k,0], initial_centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()

# サンプル番号を付ける
for n in range(num_samples):
  plt.annotate(str(n), (X_pca[n,0], X_pca[n,1]))
  
plt.show()

## ステップ4: k-meansアルゴリズムの実行  
セントロイドの初期値が作られましたので，k-meansを行う準備が整いました。  

では k-means を実行してみましょう。  
まず，ユークリッド距離を測る関数を `euclidean_distance` として定義します。  
また，k-meansによるセントロイドを1回更新する関数を `k_means` として定義します。

In [ ]:
def euclidean_distance(x, y):
  '''
      ユークリッド距離を計算して出力する
      x, y: 距離を測りたい2個の多次元ベクトル
  '''
  dist = np.sum((x-y)**2)
  dist = np.sqrt(dist)
  return dist

def k_means(centroid, data):
  '''
     k-means によるセントロイドの更新を1回行う。
     入力
     centroid: 現在のセントロイド
     data: データ
     出力
     new_centroid: 更新後のセントロイド
     assigned_class: 各データが振り分けられたクラス番号
  '''
  # クラス数を取得
  num_classes, num_dimensions = np.shape(centroid)
  # データのサンプル数と次元数を取得
  num_samples, num_dimensions = np.shape(data)

  # 各データに対して割り当てるクラス番号
  assigned_class = np.zeros(num_samples)

  # 各データに対して，最もセントロイドが近いクラスを割り当てる
  for n in range(num_samples):
    # 各クラスのセントロイドとの距離を測る
    dists = np.zeros(num_classes)
    for k in range(num_classes):
      # n番目のデータとk番目のセントロイドの距離
      dists[k] = euclidean_distance(data[n,:], centroid[k,:])

    # セントロイドとの距離が最小のクラス番号を割り当てる
    assigned_class[n] = np.argmin(dists)
  
  # セントロイドの更新
  new_centroid = np.zeros((num_classes, num_dimensions))
  for k in range(num_classes):
    # k番目のクラスに該当するデータ data[assigned_class==k] の平均値を
    # k番目のクラスの新たなセントロイドとする
    new_centroid[k] = np.mean(data[assigned_class==k,:], axis=0)
  
  return new_centroid, assigned_class

定義した関数を使って，セントロイドの更新を1回行ってみます。  

In [ ]:
centroid = initial_centroid

# k-meansによる更新を1回のみ実行
new_centroid, assigned_class = k_means(centroid, X_pca)

centroid = new_centroid

更新を1回のみ行った結果をプロットしてみましょう。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

各セントロイドと，そのセントロイドに近かったデータ，つまりそのセントロイドのクラスに割り当てられたデータが色分けされてプロットされました。

1回の更新でセントロイドがどれだけ移動したのかが分かるような形でプロットしなおしてみます。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロットの色を薄くする(alpha=0.25)
  plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1), alpha=0.25)
  # セントロイドの初期値を "x"印でプロット
  plt.scatter(initial_centroid[k,0], initial_centroid[k,1], color=c, marker='x', s=100, alpha=0.5)
  # 更新後のセントロイドを "x"印でプロット
  plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
  # セントロイドの更新前後の座標間を矢印で表示
  plt.annotate('', centroid[k], initial_centroid[k], arrowprops=dict(width=1, headwidth=8, color=c, headlength=10))
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

プロット結果から，セントロイドが移動していることが分かりました。  
セントロイドが変わっているので，再度 k-mean関数を実行して距離を測りなおせば，クラスの割り当て結果が変わり，またセントロイドの値が変わるはずです。  

ということで，k-means関数を，セントロイドが変わらなくなるまで繰り返し実行してみます。

In [ ]:
# 最大更新回数を100とする
max_iter = 100

centroid = initial_centroid

# ループの回数は max_iter とする
for n in range(max_iter):
  print('Iteration: ' + str(n+1))

  # k-meansによるセントロイドの更新を実施
  new_centroid, assigned_class = k_means(centroid, X_pca)

  # セントロイドが変わっていない，
  # つまり更新前後のセントロイドの移動距離が0の場合，ループを抜ける
  if np.sum((new_centroid - centroid)**2) == 0:
    print('centroids converged')
    break
  
  # セントロイドの更新
  centroid = new_centroid

ループを実行した結果，8回の更新でセントロイドが変わらなくなり（収束し），ループが終了しました。　　

では結果をプロットしてみましょう。

In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y']
# 二次元データのプロット
plt.figure(figsize=(10,6))
# セントロイドを"x"印でプロット 
for k in range(K):
  c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
  # 全データのプロット
  plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1))
  # セントロイドを "x"印でプロット
  plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

第5回の主成分分析のプログラム（`05_01_pca.ipynb`）によると，  
プロットの**横軸はテストの総合的な成績と関わっており，成績が良いほど左，成績が悪いほど右に向かってデータが散らばっています**。    
またプロットの**縦軸は文系理系の偏りに関わっており，理系に偏るほど上に，文系に偏るほど下に向かってデータが散らばっています**。  

従って，k-meansを用いて4クラスにクラスタリングした結果，  
* <font color='blue'>青(クラス2)：成績が良いグループ</font>  
* <font color='lightseagreen'>シアン(クラス4)：成績が悪いグループ</font> 
* <font color='red'>赤（クラス1）：総合成績は普通で，理系寄りなグループ</font>  
* <font color='green'>緑（クラス3）：総合成績は普通で，文系よりなグループ</font>  

というグループ分けが行われたと見ることができます。


## ステップ5: k-meansによる更新の様子の可視化
上記の k-means の実施結果によると，8回更新が行われて収束していました。  
この8回の更新の中で，クラス分けがどのように変化していっているのか，アニメーションで可視化してみましょう。  

以下では，k-meansの繰り返し処理を最初からやり直していますが，その際，  
更新毎のセントロイド(変数`centroid`)とクラス振り分け結果（変数`assigned_class`）の履歴を，それぞれ変数`centroid_history`と`assigned_class_history`に格納する処理を追加しています。

In [ ]:
# 最大更新回数を100とする
max_iter = 100

# 各更新におけるセントロイドおよびクラス振り分け結果の履歴を格納する変数
centroid_history = np.empty((0, K, 2), float)
assigned_class_history = np.empty((0, num_samples), int)

centroid = initial_centroid

# ループの回数は max_iter とする
for n in range(max_iter):
  print('Iteration: ' + str(n+1))

  # k-meansによるセントロイドの更新を実施
  new_centroid, assigned_class = k_means(centroid, X_pca)

  # 更新結果を履歴に追加
  centroid_history = np.append(centroid_history, np.reshape(new_centroid,[-1,K,2]), axis=0)
  assigned_class_history = np.append(assigned_class_history, np.reshape(assigned_class,[-1,num_samples]), axis=0)

  # セントロイドが変わっていない，
  # つまり更新前後のセントロイドの移動距離が0の場合，ループを抜ける
  if np.sum((new_centroid - centroid)**2) == 0:
    print('centroids converged')
    break
  
  # セントロイドの更新
  centroid = new_centroid

セントロイドとクラス振り分け情報の履歴をアニメーション表示してみます。

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

colors = ['r', 'b', 'g', 'c', 'm', 'y']

images = []
fig = plt.figure(figsize=(10,6))

# 更新回数の分だけ繰り返す
for step in range(len(centroid_history)):
  # step番目の更新時のセントロイドとクラス振り分け情報を履歴から取得する。
  assigned_class = assigned_class_history[step]
  centroid = centroid_history[step]

  # このstepのセントロイドとクラス振り分け情報をプロット
  image_tmp = []
  for k in range(K):
    c = colors[k%6] # プロットの色を，クラス番号によって自動的に変える。
    # 全データのプロット
    img = plt.scatter(X_pca[assigned_class==k,0], X_pca[assigned_class==k,1], color=c, label='class: '+str(k+1))
    image_tmp.append(img)
    # セントロイドを "x"印でプロット
    img = plt.scatter(centroid[k,0], centroid[k,1], label='centroid: '+str(k+1), color=c, marker='x', s=100)
    image_tmp.append(img)
  img = plt.text(-4.5, 2.5, 'Iteration: '+str(step+1), size='x-large')
  image_tmp.append(img)

  images.append(image_tmp)
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=1000)

# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

セントロイドが移動しながら，各データが所属するクラスの情報も変わっていき，最終的なクラスタリング結果に収束する様子が分かります。